# User input; only parameter

In [153]:
user_input="C:/Users/A02315404/Downloads/Navajo_Knobs.gpx"
#user_input="C:/Users/A02315404/Downloads/Temple_Fork.gpx"
#user_input = "C:/Users/A02315404/Downloads/GPS tracks/Little_Black_Mountain_trail_run_with_Matt.gpx"

# Importing modules, naming data folders

In [154]:
#modules (environ = raster_proj)
import arcpy
import os
#import classtools
arcpy.CheckOutExtension('spatial')
from arcpy import env
from arcpy.sa import *
import rasterio
import csv
import numpy as np
import gdal

arcpy.overwriteoutput = True

filename = os.path.split(user_input)[1]
filename = filename.replace(".gpx", "")

#folders
conv_data_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\data_converted'
data_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\data'
data_orig_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\data_orig'
int_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\intersections'
in_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\data_gpx_files'
out_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\data_converted'
frolic_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis'

arcpy.env.workspace = data_folder

#shape/image data
rock = os.path.join(data_orig_folder, "rock.tif")
nlcd = os.path.join(data_orig_folder, "nlcd.tif")
nlcd_key = os.path.join(data_orig_folder, "nlcd_key.csv")
rock_key = os.path.join(data_orig_folder, "rock_key.csv")
eco3 = os.path.join(data_orig_folder, "ecoregionsL3.shp")
eco4 = os.path.join(data_orig_folder, "ecoregionsL4.shp")


#trail (from user input)
trail =  os.path.join(conv_data_folder, filename) + ".shp"
output_name_shp = os.path.join(out_folder, filename)+ ".shp"
output_name_tif = os.path.join(out_folder, filename)+ ".tif"
out_name_nlcd=os.path.join(int_folder, filename) + "_nlcd.tif"
out_name_rock=os.path.join(int_folder, filename) + "_rock.tif"
temp_fn = os.path.join(frolic_folder, filename) + '_test_maps.pdf'
pdf_fn = os.path.join(frolic_folder, filename) + '_maps.pdf'


# Converting user_input (.gpx) to .tif and  .shp files

In [155]:
#print(filename,"\n", output_name_shp, "\n", output_name_tif)
arcpy.GPXtoFeatures_conversion(user_input, output_name_shp)
arcpy.conversion.PointToRaster(in_features=output_name_shp, 
                               value_field='FID', 
                               out_rasterdataset=output_name_tif)


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000725: Output Feature class: Dataset C:\Users\A02315404\Documents\python\frolic_analysis\data_converted\Navajo_Knobs.shp already exists.
Failed to execute (GPXtoFeatures).


# Raster Stats 

In [156]:
#make dictionaries from csv files
nlcd_dict = {}
with open(nlcd_key, mode='r') as inp:
    reader = csv.reader(inp)
    nlcd_dict = {rows[0]:rows[1] for rows in reader}

rock_dict = {}
with open(rock_key, mode='r') as inp:
    reader = csv.reader(inp)
    rock_dict = {rows[0]:rows[1] for rows in reader}

#define output intersections    
data_list = [trail,rock,nlcd]
out_name=os.path.join(int_folder, filename) + ".tif"

#use ExtractByMask to make intersections
outExtractByMask=ExtractByMask(in_raster=nlcd, in_mask_data=trail)
outExtractByMask.save(out_name_nlcd)
outExtractByMask=ExtractByMask(in_raster=rock, in_mask_data=trail)
outExtractByMask.save(out_name_rock)

#get unique values from rasters
rock_np=gdal.Open(out_name_rock).GetRasterBand(1)
rock_min = str(round(Raster(out_name_rock).minimum))
rock_max = str(round(Raster(out_name_rock).maximum))
rock_vals =np.unique(np.array([rock_min, rock_max]))
nlcd_np=gdal.Open(out_name_nlcd).GetRasterBand(1)
nlcd_min = str(round(Raster(out_name_nlcd).minimum))
nlcd_max = str(round(Raster(out_name_nlcd).maximum))
nlcd_vals =np.unique(np.array([nlcd_min, nlcd_max]))

#print results
for r in rock_vals:
    print(" You hiked in the", rock_dict[str(r)], "zone of lithology!")
for n in nlcd_vals:
    print(" You hiked in the", nlcd_dict[str(n)], "landcover classification zone!")



RuntimeError: ERROR 000872: Output Raster: Dataset C:\Users\A02315404\Documents\python\frolic_analysis\intersections\Navajo_Knobs_nlcd.tif already exists and cannot be overwritten since the Overwrite existing datasets option is disabled.

# Shp Stats

In [157]:
#make intersection
data_list = [trail,eco3,eco4]
out_name=os.path.join(int_folder, filename) + ".shp"
print(out_name)
arcpy.analysis.Intersect(in_features=data_list, 
                         out_feature_class=out_name, 
                         )
#funtion to get unique fieldnames
def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

eco4_vals = unique_values(out_name , 'us_l4name')
eco3_vals =unique_values(out_name , 'na_l3name')
#make dictionaries of results
eco4_dict = {}
eco3_dict = {}
count_all = int(arcpy.management.GetCount(out_name).getOutput(0))
count=0
for eco4_name in eco4_vals:
    subset_eco4 = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view= out_name, 
        where_clause=f"us_l4name LIKE  '{eco4_vals[count]}' "
    )
    percent_eco4 = ((int(arcpy.management.GetCount(subset_eco4).getOutput(0)))/int(count_all)*100 )
    eco4_dict[eco4_vals[count]] = percent_eco4
    count=count+1
count=0
for eco3_name in eco3_vals:
    subset_eco3 = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view= out_name, 
        where_clause=f"na_l3name LIKE  '{eco3_vals[count]}' "
    )
    percent_eco3 = ((int(arcpy.management.GetCount(subset_eco3).getOutput(0)))/int(count_all)*100 )
    eco3_dict[eco3_vals[count]] = percent_eco3
    count=count+1
#print results
count=0
for eco in eco3_dict.items():
    if count==0:
        print(f"Your adventure took you through the {eco[0]} ( {round(eco[1])} % ) ") 
    if count!=0:
        print(f"and the {eco[0]} ( {round(eco[1])} % ) ") 
    count=count+1
count=0
for eco in eco4_dict.items():
    if count==0:
        print(f"specifically through the {(eco[0])} ( {round(eco[1])} % ) ")
    if count!=0:
        print(f" and the {(eco[0])} ( {round(eco[1])} % ) ") 
    count=count+1


C:\Users\A02315404\Documents\python\frolic_analysis\intersections\Navajo_Knobs.shp


ExecuteError: Failed to execute. Parameters are not valid.
WARNING 001003: Datum conflict between input and output.
ERROR 000725: Output Feature Class: Dataset C:\Users\A02315404\Documents\python\frolic_analysis\intersections\Navajo_Knobs.shp already exists.
Failed to execute (Intersect).


# Make maps

- getting current map, layouts, and naming base data layers and user input layer

In [159]:
# Get the current project.
import arcpy
project = arcpy.mp.ArcGISProject("C:/Users/A02315404/Documents/python/frolic_analysis/frolic_map1.aprx")
[m.name for m in project.listMaps()]
# Get the currently active map.
mapp = project.listMaps('utah')[0]
# Get the  layers.
eco3 = mapp.listLayers('eco3_crop')[0]
eco4 = mapp.listLayers('eco4_crop')[0]
rock = mapp.listLayers('rock_crop.tif')[0]
nlcd = mapp.listLayers('nlcd_crop.tif')[0]
lyr_list = [eco3,eco4,rock,nlcd]
lyr_names = ["Ecoregion Classificaiton III","Ecoregion Classificaiton IV","Type of Lithology" , "Landcover"]
mapp.addDataFromPath(trail)
trail_lyr = mapp.listLayers(filename)[0]
mapp = project.listMaps('utah')[0]
layout = project.listLayouts('current_layout')[0]
final_doc = arcpy.mp.PDFDocumentCreate(pdf_fn)

# Making and saving pdf of maps


In [160]:
#0
print(0)
cur_lyr = lyr_list[0]
#mapp.addLayer(cur_lyr, "TOP")
print("cur_lyr:", cur_lyr)


lyr_list = [eco4,rock,nlcd]

for lyr in lyr_list:
    if (lyr != cur_lyr):
        lyr.visible=False

cur_lyr.visible=True
trail_lyr.visible=True

layout.exportToPDF(temp_fn)
final_doc.appendPages(temp_fn)
cur_lyr.visible=False
mapp.removeLayer(cur_lyr)


#1
print(1)
cur_lyr = lyr_list[0]
#mapp.addLayer(cur_lyr, "TOP")
print("cur_lyr:", cur_lyr)
lyr_list = [rock,nlcd]

for lyr in lyr_list:
    if (lyr != cur_lyr):
        lyr.visible=False

cur_lyr.visible=True
trail_lyr.visible=True

layout.exportToPDF(temp_fn)
final_doc.appendPages(temp_fn)
cur_lyr.visible=False
mapp.removeLayer(cur_lyr)


#2
print(2)
cur_lyr = lyr_list[0]
#mapp.addLayer(cur_lyr, "TOP")
print("cur_lyr:", cur_lyr)
lyr_list = [nlcd]
for lyr in lyr_list:
    if (lyr != cur_lyr):
        lyr.visible=False

cur_lyr.visible=True
trail_lyr.visible=True

layout.exportToPDF(temp_fn)
final_doc.appendPages(temp_fn)
cur_lyr.visible=False
mapp.removeLayer(cur_lyr)


#3
print(3)
cur_lyr = lyr_list[0]
#mapp.addLayer(cur_lyr, "TOP")
print("cur_lyr:", cur_lyr)
for lyr in lyr_list:
    if (lyr != cur_lyr):
        lyr.visible=False


cur_lyr.visible=True
trail_lyr.visible=True

layout.exportToPDF(temp_fn)
final_doc.appendPages(temp_fn)

# Save and close the final pdf.
final_doc.saveAndClose()

# Delete variables to make sure everything is closed.
del final_doc, project

0
cur_lyr: eco3_crop
1
cur_lyr: eco4_crop
2
cur_lyr: rock_crop.tif
3
cur_lyr: nlcd_crop.tif
